# 18. 딥러닝 들여다보기

## 1. 들어가며

### 학습 목표
---
1. 딥러닝 문제 구성에 대한 기본적인 이해를 높인다.
2. Neural Network에 사용되는 용어들에 대한 이해를 높인다.
3. 딥러닝 프레임워크를 사용하지 않고, Numpy만을 이용해 딥러닝 모델과 훈련 과정을 직접 구현해 본다.

### 오늘은 어떤 걸 배울까?
---
그동안 딥러닝을 이용한 몇 가지 프로젝트를 수행해 보고, 관련하여 딥러닝 프레임워크를 활용해 보면서 딥러닝의 문제 구성이 어떠한지 대략적인 흐름을 익혔다.
그러나 딥러닝 프레임워크 안쪽에서 실제로 어떤 메커니즘으로 모델이 학습되는지 그 디테일까지 명확하게 파악하지는 못해서 나름의 답답함을 느꼈다.

오늘은 딥러닝 내부를 좀 더 깊게 들여다보겠다. 신경망(Neural Network)이 어떤 식으로 구성되어 있고, 그 과정에서 어떤 용어들이 사용되는지 공부할 것이다. 그 과정에서 그동안 활용해 왔던 딥러닝 프레임워크를 사용하지 않고, 오직 Numpy만을 이용해 간단한 신경망과 그 훈련 과정을 직접 구현해 볼 것이다.

## 2. 신경망 구성 (1) 개요
신경망(Neural Network)이란 무엇일까?

우리 뇌에는 `1000억 개`에 가까운 신경계 뉴런들이 있다고 한다. 이 뉴런들은 서로 매우 복잡하게 얽혀 있고, 조금 물러서서 보면 하나의 거대한 그물망과 같은 형태를 이루고 있다. 보통 이를 신경망이라고 부른다.

머신러닝/딥러닝 과학자들도 자연에서 답을 찾으려 노력했고, 우리 뇌 속의 신경망 구조에 착안해서 퍼셉트론(Perceptron)이라는 형태를 제안하며 이를 연결한 형태를 인공신경망(Artificial Neural Network)이라고 부르기 시작했다.

### MNIST Revisited
---
MNIST 이미지 분류기 모델이 어떻게 구성되었는지 기억을 되짚어 보자. 여러번 구현해 보았던 Tensorflow 기반 분류 모델 예시 코드이다.



In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# MNIST 데이터를 로드. 다운로드하지 않았다면 다운로드까지 자동으로 진행됩니다. 
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 모델에 맞게 데이터 가공
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped = x_train_norm.reshape(-1, x_train_norm.shape[1]*x_train_norm.shape[2])
x_test_reshaped = x_test_norm.reshape(-1, x_test_norm.shape[1]*x_test_norm.shape[2])

# 딥러닝 모델 구성 - 2 Layer Perceptron
model=keras.models.Sequential()
model.add(keras.layers.Dense(50, activation='sigmoid', input_shape=(784,)))  # 입력층 d=784, 은닉층 레이어 H=50
model.add(keras.layers.Dense(10, activation='softmax'))   # 출력층 레이어 K=10
model.summary()

# 모델 구성과 학습
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train_reshaped, y_train, epochs=10)

# 모델 테스트 결과
test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4993 - accuracy: 0.8840
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2287 - accuracy: 0.9358
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1786 - accuracy: 0.9499
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1487 - accuracy: 0.9581
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1280 - accuracy: 0.9636
Epoc

가장 기본적인 신경망 형태인 다층 퍼셉트론(Multi-Layer Perceptron; MLP)만을 이용해서 더욱 간단하게 구현해 보았다. 뇌 속의 뉴런이 1000억 개라지만 위에서 만든 모델은 굳이 Conv2D를 사용하지도 않았는데도 39,760개의 파라미터만으로 테스트 성능이 97%에 육박하고 있다.<br>
인공신경망의 실제 구현 원리를 보다 명확하게 이해하기 위해, 그동안 들춰보지 않았던 프레임워크 내부에서 일어나는 일을 Numpy를 활용해 직접 구현해 보면서 이해해 보고자 한다.

### 다층 퍼셉트론 Overview
---
위의 이미지는 총 3개의 레이어로 구성 된 퍼셉트론을 나타낸다. 은닉층에는 `H개의 노드`가, 출력층에는 `K개의 노드`가 존재하는 인공신경망을 표현한 것이다. (+1 부분은 bias를 뜻하는 부분이므로 이전 레이어와의 연결이 없다.) 위의 코드에서는 `H=50`, `K=10`, 그리고 입력층 노드 개수 d=784로 정의되었다.

bias 노드에 대해서는 아래 참고자료.

`참고자료: bias term`
* [The Basic Artificial Neuron: Bias neuron(Backpropagation)](https://funnypr.tistory.com/entry/The-Basic-Artificial-Neuron-Bias-neuron)
* [What is the role of the bias in neural networks?](https://stackoverflow.com/questions/2480650/what-is-the-role-of-the-bias-in-neural-networks)

이제 각 레이어에 대해 한번 알아보자.

위의 이미지를 보면 입력값이 있는 입력층(input layer), 최종 출력값이 있는 출력층(output layer), 그리고 그 사이에 있는 층인 은닉층(hidden layer)이 있다. 보통 입력층과 출력층 사이에 몇 개의 층이 존재하든 모두 은닉층이라고 부른다.

보통 그림으로 인공신경망을 표현할 때에는 노드를 기준으로 레이어를 표시해서 3개의 레이어라고 생각할 수 있지만, 실제로는 `총 2개의 레이어`를 가졌다. 레이어 개수를 셀 때는 노드와 노드 사이의 연결하는 부분이 몇 개 존재하는지 세면 보다 쉽게 알 수 있다.

이렇게 인공신경망이 어떻게 생겼는지 대략 알아보았다. 인공신경망 중에서도 위의 이미지처럼 2개 이상의 레이어를 쌓아서 만든 것을 보통 `다층 퍼셉트론(Multi-Layer Perceptron; MLP)`이라고 부른다. 그리고 입력층, 출력층을 제외한 은닉층이 많아지면 많아질수록 인공신경망이 `DEEP` 해졌다고 이야기한다.

지금 알아보려고 하는 딥러닝이 바로 이 인공신경망이 `DEEP`해졌다는 뜻에서 나온 단어이다. 그래서 딥러닝은 충분히 깊은 인공신경망을 활용하며 이를 보통 다른 단어로 `DNN(Deep Neural Network)`이라고 부른다.

`Tips`

>*💡Fully-Connected Neural Network와 같은 단어를 들어보았을 것이다 이는 앞에서 설명한 MLP의 다른 용어이다. 이 Fully-Connnected Nerual Network는 서로 다른 층에 위치한 노드 간에는 연결 관계가 존재하지 않으며, 인접한 층에 위치한 노드들 간의 연결만 존재한다는 의미를 내포한다.*

### Parameters/Weights
---
앞에서 설명한 입력층-은닉층, 은닉층-출력층 사이에는 사실 각각 행렬(Matrix)이 존재한다. 고등학교 때 나오는 행렬 곱셈을 사용해보자.

예를 들어 입력값이 100개, 은닉 노드가 20개라면 사실 이 입력층-은닉층 사이에는 100x20의 형태를 가진 행렬이 존재한다. 똑같이, MNIST 데이터처럼 10개의 클래스를 맞추는 문제를 풀기 위해 출력층이 10개의 노드를 가진다면 은닉층-출력층 사이에는 20x10의 형태를 가진 행렬이 존재하게 된다.

이 행렬들을 Parameter 혹은 Weight라고 부른다. 두 단어는 보통 같은 뜻으로 사용되지만, 실제로 Paraemter에는 위의 참고자료에서 다룬 bias 노드도 포함된다는 점 유의해야 한다.

이때 인접한 레이어 사이에는 아래와 같은 관계가 성립한다.

$y = W * X + b $

간단히 만들어 보았던 MLP 기반 딥러닝 모델을 Numpy로 다시 만들어 보자.

In [ ]:
# 입력층 데이터의 모양(shape)
print(x_train_reshaped.shape)

# 테스트를 위해 x_train_reshaped의 앞 5개의 데이터를 가져온다.
X = x_train_reshaped[:5]
print(X.shape)

In [3]:
weight_init_std = 0.1
input_size = 784
hidden_size=50

# 인접 레이어간 관계를 나타내는 파라미터 W를 생성하고 random 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size)  
# 바이어스 파라미터 b를 생성하고 Zero로 초기화
b1 = np.zeros(hidden_size)

a1 = np.dot(X, W1) + b1   # 은닉층 출력

print(W1.shape)
print(b1.shape)
print(a1.shape)

(784, 50)
(50,)
(5, 50)


In [4]:
# 첫 번째 데이터의 은닉층 출력을 확인해 봅시다.  50dim의 벡터가 나오나요?
a1[0]

array([-0.66812842,  1.21796762,  1.07363698, -0.09432575,  0.0760824 ,
        0.30708839, -0.63175526,  0.58774867, -1.02994591, -0.04817563,
        1.18097436, -1.05733832, -1.82666649, -1.0094017 , -0.02244052,
       -0.27265812, -0.68179908,  1.82527517, -0.19865562,  0.77474247,
        0.11511811,  1.51474221,  0.19132956, -1.39466579,  0.48986256,
        0.83423317, -1.71459918, -1.8022644 ,  1.16650174,  0.78464758,
        0.0132319 ,  0.09772938,  0.28977091, -1.161101  ,  0.29028108,
        0.43745435, -0.8172701 , -0.00253478,  0.62268625, -0.61532497,
        1.46014522, -0.23094997, -1.80560602, -2.09141006, -0.96998288,
        1.61527891, -0.4723759 ,  0.47537838, -1.59787021, -0.65766017])

## 3. 신경망 구성 (2) 활성화 함수와 손실 함수

### 활성화 함수 (Activation Functions)
---
MLP의 또 다른 중요한 구성요소에는 활성화 함수가 있다. 딥러닝에서는 이 활성화 함수의 존재가 필수적이다. 수학적인 이유가 있지만, 간단히만 설명하자면 이 활성화 함수는 보통 비선형 함수를 사용하는데 이 비선형 함수를 MLP 안에 포함시키면서 모델의 표현력이 좋아지게 된다. (정확히는 레이어 사이에 이 비선형 함수가 포함되지 않은 MLP는 한개의 레이어로 이루어진 모델과 수학적으로 다른 점이 없다.)

(참고) [Why must a nonlinear activation function be used in a backpropagation neural network?](https://stackoverflow.com/questions/9782071/why-must-a-nonlinear-activation-function-be-used-in-a-backpropagation-neural-net/54503251#54503251)

그럼 이렇게 중요한 역할을 하는 활성화 함수에는 어떤 종류가 있을까? 몇 가지 자주 쓰이는 활성화 함수들에 대해서 알아보도록 하자.

#### 1. Sigmoid
$\sigma (\mathit{x}) = \frac{1}{1 + e^{-x}}$